# Word Cloud - Tweeter

In [1]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/tlwg/Laksaman.ttf'


from IPython import display
import time

from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession.builder\
    .master('local[*]')\
    .appName('WordCloud')\
    .getOrCreate()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/franciscofoz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


23/05/23 20:46:23 WARN Utils: Your hostname, franciscofoz-desktop resolves to a loopback address: 127.0.1.1; using 192.168.0.12 instead (on interface enp2s0)
23/05/23 20:46:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/23 20:46:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
def trata_tweets(df):
    words = df \
        .select(f.explode(f.split(f.lower('_c0'), " ")).alias("word")) \
        .withColumn('word', f.regexp_replace('word', r'http\S+', '')) \
        .withColumn('word', f.regexp_replace('word', r'@\w+', '')) \
        .withColumn('word', f.regexp_replace('word', 'rt', '')) \
        .na.replace('', None) \
        .na.drop()
    return words


In [ ]:
stops = stopwords.words('portuguese')
stops.append('biblioteconomia')
plt.figure(figsize=(20, 10))

while True:
    try:
        words = spark.read.csv('./csv', encoding='latin1')
        words = trata_tweets(words)
        rows = words.collect()
        all_words = ''
        for row in rows:
            all_words = all_words + ' ' + row['word']

        wordcloud = WordCloud(stopwords=stops,
                              background_color="black",
                              width=1920,
                              height=1080,
                              max_words=100,
                              font_path=font_path
                              ).generate(all_words)

        plt.cla()
        plt.axis('off')
        plt.imshow(wordcloud)
        display.display(plt.gcf())
        display.clear_output(wait=True)
        time.sleep(5)
    except KeyboardInterrupt:
        break

In [ ]:
import matplotlib.font_manager as fm

font_list = fm.findSystemFonts()
for font in font_list:
    print(font)
